# Process
1. 랜덤으로 700명 추출
2. Base Loss 측정
3. 각 피처마다 0 -> 1, 1 -> 0으로 바꾸어서 모델 train, predict 후 loss 측정
4. 비교 DF 생성

In [1]:
import numpy as np, os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [25]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

X_700 = X[idx[:700]]
y_700 = y[idx[:700]]
print(X_700.shape)

i = round(X_700.shape[0]*0.8)
X_train, y_train = X_700[:i,:,:], y_700[:i]
X_test, y_test = X_700[i:,:,:], y_700[i:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(700, 10, 4069)


((560, 10, 4069), (560,), (140, 10, 4069), (140,))

In [28]:
def get_model():
    seed_num = 48
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(X.shape[1],X.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate = 0.001)
    lstm.compile(optimizer=optimizer, loss = "binary_crossentropy", metrics=['acc'])
    return lstm

# Base Loss
- 916.48851128

In [29]:
model = get_model()
early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop], shuffle=False)

Epoch 1/500
4/4 [==============================] - 3s 163ms/step - loss: 0.6796 - acc: 0.6048 - val_loss: 0.6735 - val_acc: 0.6000
Epoch 2/500
4/4 [==============================] - 0s 58ms/step - loss: 0.6637 - acc: 0.6071 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 3/500
4/4 [==============================] - 0s 58ms/step - loss: 0.6779 - acc: 0.6048 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 4/500
4/4 [==============================] - 0s 56ms/step - loss: 0.6732 - acc: 0.6048 - val_loss: 0.6732 - val_acc: 0.6000
Epoch 5/500
4/4 [==============================] - 0s 55ms/step - loss: 0.6724 - acc: 0.6024 - val_loss: 0.6732 - val_acc: 0.6000
Epoch 6/500
4/4 [==============================] - 0s 55ms/step - loss: 0.6738 - acc: 0.6095 - val_loss: 0.6733 - val_acc: 0.6000
Epoch 7/500
4/4 [==============================] - 0s 54ms/step - loss: 0.6714 - acc: 0.6071 - val_loss: 0.6731 - val_acc: 0.6000
Epoch 8/500
4/4 [==============================] - 0s 55ms/step - loss: 0.6732 - acc: 0.6

In [65]:
def BinaryCrossEntropy(y_true, y_pred):
    m = y_true.shape[1]
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Calculating loss
    loss = -1/m * (np.dot(y_true.T, np.log(y_pred)) + np.dot((1 - y_true).T, np.log(1 - y_pred)))

    return loss

In [97]:
pred = model.predict(X_test)
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
BinaryCrossEntropy(pred, y_test)[0]

916.4885112753052

# Reversed Feature
- https://stackoverflow.com/questions/39164786/invert-0-and-1-in-a-binary-array

In [ ]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

X_700 = X[idx[:700]]
y_700 = y[idx[:700]]
print(X_700.shape)

i = round(X_700.shape[0]*0.8)
X_train, y_train = X_700[:i,:,:], y_700[:i]
X_test, y_test = X_700[i:,:,:], y_700[i:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [68]:
X_700.shape, y_700.shape

((700, 10, 4069), (700,))

In [95]:
# 컬럼이름 로드
COLS = list(pd.read_csv('/project/guri/data/data2d_forposter.csv').columns)

In [140]:
a = [1,0,0,1,0,1]
def reverse(li):
    return np.logical_not(li).astype(int)

reverse(a)

array([0, 1, 1, 0, 1, 0])

In [143]:
results = []
result.append({'feaure':'BASELINE', 'bce':916.4885112753052})
for i in tqdm(range(len(COLS))):
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,:,i] = reverse(X_test[p,:,i])
        
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})


SyntaxError: invalid syntax (<ipython-input-143-ce6f03a539ed>, line 11)

In [142]:
df = pd.DataFrame(results)
df = df.sort_values('bce')
df

,feature,bce
4,50806,934.737265
2,50804,936.055653
0,0,936.068178
3,50805,936.107944
1,50803,938.012941
...,...,...
3834,63323026530,1054.132033
3838,63323027810,1054.132173
3943,65649023141,1054.135837
3941,65597070118,1054.142382


# ALL (7727)

In [144]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[:i,:,:], y[:i]
X_test, y_test = X[i:,:,:], y[i:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [145]:
model = get_model()
early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 65ms/step - loss: 0.6255 - acc: 0.6872 - val_loss: 0.7495 - val_acc: 0.4955
Epoch 2/500
37/37 [==============================] - 2s 53ms/step - loss: 0.6240 - acc: 0.6887 - val_loss: 0.7583 - val_acc: 0.4955
Epoch 3/500
37/37 [==============================] - 2s 53ms/step - loss: 0.6265 - acc: 0.6885 - val_loss: 0.7581 - val_acc: 0.4955
Epoch 4/500
37/37 [==============================] - 2s 53ms/step - loss: 0.5813 - acc: 0.6943 - val_loss: 0.6175 - val_acc: 0.6953
Epoch 5/500
37/37 [==============================] - 2s 53ms/step - loss: 0.4594 - acc: 0.7871 - val_loss: 0.5644 - val_acc: 0.7277
Epoch 6/500
37/37 [==============================] - 2s 53ms/step - loss: 0.3944 - acc: 0.8281 - val_loss: 0.5836 - val_acc: 0.7212
Epoch 7/500
37/37 [==============================] - 2s 53ms/step - loss: 0.3458 - acc: 0.8598 - val_loss: 0.6110 - val_acc: 0.7342
Epoch 8/500
37/37 [==============================] - 2s 53ms/step - loss: 0.

In [146]:
pred = model.predict(X_test)
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
base_bce = BinaryCrossEntropy(pred, y_test)[0]

In [148]:
results = []
results.append({'feature':'BASELINE', 'bce':base_bce})

for i in tqdm(range(len(COLS))):
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,:,i] = reverse(X_test[p,:,i])
        
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})

  0%|          | 0/4069 [00:00<?, ?it/s]

In [151]:
results.append({'feature':'BASELINE', 'bce':base_bce})

In [153]:
df = pd.DataFrame(results)
df = df.sort_values('bce')
df1 = df.drop('feaure', axis=1)
df1

,bce,feature
2,7985.298345,50803
0,8008.864772,NaN
4070,8008.864772,BASELINE
3,8010.250480,50804
1,8058.704548,0
...,...,...
648,12966.786711,9005605
626,12966.833315,8103007
642,12968.977497,9001104
637,12970.154025,8499002


In [154]:
df1.to_csv('./reverse_FI.csv', index=False)
a = pd.read_csv('./reverse_FI.csv')

In [159]:
a[-10:]

,bce,feature
4061,12962.064694,9000602
4062,12964.295344,8536002
4063,12964.877631,8499020
4064,12964.930786,9006412
4065,12966.520841,8103006
4066,12966.786711,9005605
4067,12966.833315,8103007
4068,12968.977497,9001104
4069,12970.154025,8499002
4070,12971.108235,8103105


# ALL (7727) - D-0~4

In [ ]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[:i,:,:], y[:i]
X_test, y_test = X[i:,:,:], y[i:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
model = get_model()
early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop], shuffle=False)

In [ ]:
pred = model.predict(X_test)
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
base_bce = BinaryCrossEntropy(pred, y_test)[0]

In [ ]:
results = []
results.append({'feature':'BASELINE', 'bce':base_bce})

for i in tqdm(range(len(COLS))):
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,:,i] = reverse(X_test[p,:,i])
        
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})